In [50]:
import gpytorch
import torch
import numpy as np

In [51]:


train_x = torch.Tensor([[0, 0, 0, 0,], [1, 1, 1, 1]])
train_y = torch.Tensor([0.1, 0.1])
s = 2
l = [1.5, 1.5, 1.5, 1.5]
epsilon = 0.05

In [52]:
# We will use the simplest form of GP model, exact inference
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ZeroMean()
        self.covar_module = gpytorch.kernels.RBFKernel(ard_num_dims=4)

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(train_x, train_y, likelihood)

In [53]:
def f4(x):
    return x + torch.log(-torch.expm1(-x))


model.covar_module.raw_lengthscale.data = torch.Tensor([f4(torch.tensor(1.5)), f4(torch.tensor(1.5)), f4(torch.tensor(1.5)), f4(torch.tensor(1.5)) ])
model.likelihood.raw_noise.data = torch.tensor([f4(torch.tensor(epsilon))])

list(model.named_parameters())

[('likelihood.noise_covar.raw_noise',
  Parameter containing:
  tensor([-2.9706], requires_grad=True)),
 ('covar_module.raw_lengthscale',
  Parameter containing:
  tensor([1.2475, 1.2475, 1.2475, 1.2475], requires_grad=True))]

In [54]:
# Get into evaluation (predictive posterior) mode
model.eval()
likelihood.eval()

# Test points are regularly spaced along [0,1]
# Make predictions by feeding model through likelihood
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    output = model(train_x)
    observed_pred = likelihood(output)
    

/home/besginow/anaconda3/envs/sage/lib/python3.10/site-packages/gpytorch/models/exact_gp.py:274: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  warnings.warn(


In [56]:
observed_pred.covariance_matrix

tensor([[0.0974, 0.0011],
        [0.0011, 0.0974]])

In [57]:
observed_pred.loc

tensor([0.0966, 0.0966])